In [52]:
import re
class HashableDict(dict):
    def __hash__(self):
        return hash(frozenset(self.items()))
    
    
starting_positions = HashableDict()
with open("input.txt", "r") as f:
    for l in f.read().splitlines():
        player, pos = re.match("Player (\d) starting position: (\d+)", l).groups()
        starting_positions[int(player)] = int(pos)
        
players = HashableDict()
for player , position in starting_positions.items():
    players[player] = (position, 0)

playing_player = 1  
iter = 0
rolls = 0
finished = False
while not finished:
    dice = (iter * 3) % 100 + 1 
    if dice <=98:
        r = 3*dice+3
    else:
        n1 = dice
        new_dice = dice + 1
        n2 = new_dice if new_dice <= 100 else new_dice - 100
        new_dice = dice + 2
        n3 = new_dice if new_dice <= 100 else new_dice - 100
        r = n1 + n2 + n3  
    rolls += 3      
    pos, score = players[playing_player]
    new_pos = (pos + r - 1) % 10 + 1
    new_score = score + new_pos
    #print(f"iter: {iter}, dice: {dice}, player:{playing_player}, score:{new_score}")
    if (new_score >= 1000):
        losing_score = players[2 if playing_player == 1 else 1][1]
        #print(f"iter: {iter+1}, rolls: {rolls}, player: {playing_player}, losing_score: {losing_score}")
        print(f"answer_1: {losing_score * rolls}")
        finished = True
    players[playing_player] = (new_pos, new_score)
    iter += 1
    playing_player = 2 if playing_player == 1 else 1
    #print(players)
         
    

answer_1: 995904


In [54]:
from collections import Counter, defaultdict
from itertools import groupby
import math
    
p = defaultdict(lambda : 0)
for d1 in range(1,4):
    for d2 in range(1,4):
        for d3 in range(1,4):
            s = d1+d2+d3
            p[s] += 1

n = 3**3
p = dict(map(lambda x: (x[0], x[1]/n), p.items()))        
pp = list()
for o1, p1 in p.items():
    for o2, p2 in p.items():
        pp.append((o1, o2, p1*p2))
        
players = HashableDict()
for player , position in starting_positions.items():
    players[player] = (position, 0)
    
def play_round(universes, pp):
    outcomes = dict()
    wins = []
    for board, universe in universes.items():
        n_round = universe[1]+1
        for d1, d2, p in pp:
            p_universe = universe[0]*p
            #print(n_round, p_universe)
            pos1, score1 = board[1]
            new_pos1 = (pos1 + d1 - 1) % 10 + 1
            new_score1 = score1 + new_pos1
            pos2, score2 = board[2]
            new_pos2 = (pos2 + d2 - 1) % 10 + 1
            new_score2 = score2 + new_pos2
            if new_score1 >= 21:
                wins.append((1, p_universe, n_round))
            elif new_score2 >= 21:
                wins.append((2, p_universe, n_round))
            else:
                score = HashableDict({1: (new_pos1, new_score1), 2: (new_pos2, new_score2)})
                if score not in outcomes:
                    outcomes[score]= (p_universe, n_round)
                else:
                    outcomes[score] =(outcomes[score][0] + p_universe, n_round)
    return outcomes, wins

universes = {players: (1, 0)}
all_wins = []
while len(universes)>0:
    universes, wins = play_round(universes, pp)
    all_wins.extend(wins)

count_universes = Counter()
tot_prob = 0
player1 = []
player2 = []
player1_i = []
player2_i = []
i=0
for player, prob, n_round in all_wins:
    i += 1
    count_universes[player] += prob * ((27*27)**n_round if player==2 else (27*27)**(n_round-1)*27)
    if player==1:
        player1_i.append(i)
        player1.append(prob * (27*27)**n_round)
    else:
        player2_i.append(i)
        player2.append(prob * (27*27)**n_round)
    tot_prob += prob    

answer_2 = round(count_universes.most_common()[0][1])
print(f"answer_2: {answer_2}")


answer_2: 193753136998081
